In [4]:
!nvidia-smi

Fri Jun 13 04:49:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          Off |   00000000:06:10.0 Off |                    0 |
| N/A   38C    P0            274W /  700W |   77498MiB /  81559MiB |     87%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [1]:
from mcp.server.fastmcp import FastMCP
import arxiv
import asyncio
from pathlib import Path
import os

In [5]:
print(Path.home())

/home/siamai


In [2]:
mcp = FastMCP("ArXiv Research Server")
# Storage setup
STORAGE_PATH = Path.home() / "deepsad" / "local-deep-researcher" /"notebooks" /".arxiv-mcp" / "papers"
STORAGE_PATH.mkdir(parents=True, exist_ok=True)
print(f"✓ Created ArXiv MCP server")
print(f"✓ Storage path: {STORAGE_PATH}")

✓ Created ArXiv MCP server
✓ Storage path: /home/siamai/deepsad/local-deep-researcher/notebooks/.arxiv-mcp/papers


### Tool 1: Search Paper

In [3]:
@mcp.tool()
def search_papers(query: str, max_results: int = 10) -> str:
    try:
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )
        
        papers = []
        for result in arxiv.Client().results(search):
            papers.append({
                "id": result.entry_id.split("/")[-1],
                "title": result.title,
                "authors": [author.name for author in result.authors],
                "summary": result.summary[:200] + "...",
                "published": result.published.isoformat(),
                "categories": result.categories
            })
        
        # Return actual paper details, not just status message
        if not papers:
            return "No papers found for query: " + query
            
        result_text = f"Found {len(papers)} papers:\n\n"
        for i, p in enumerate(papers, 1):
            result_text += f"{i}. **{p['title']}**\n"
            result_text += f"   ID: {p['id']}\n"
            result_text += f"   Authors: {', '.join(p['authors'][:3])}\n"
            result_text += f"   Published: {p['published'][:10]}\n"
            result_text += f"   Summary: {p['summary']}\n\n"
        
        return result_text
        
    except Exception as e:
        return f"Search error: {str(e)}"

# Test with the fixed function
test_result = search_papers("reasoning model for agentic system", 5)
print(test_result)
    
print("✓ Added search_papers tool")

[06/13/25 09:33:39] INFO     Requesting page (first: True, try: 0):                                 ]8;id=545527;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=617137;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#680\680]8;;\
                             https://export.arxiv.org/api/query?search_query=reasoning+model+for+ag                
                             entic+system&id_list=&sortBy=relevance&sortOrder=descending&start=0&ma                
                             x_results=100                                                                         

[06/13/25 09:33:43] INFO     Got first page: 100 of 2442103 total results                           ]8;id=718847;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=602671;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#606\606]8;;\

Found 5 papers:

1. **Agentic Reasoning: Reasoning LLMs with Tools for the Deep Research**
   ID: 2502.04644v1
   Authors: Junde Wu, Jiayuan Zhu, Yuyuan Liu
   Published: 2025-02-07
   Summary: We introduce Agentic Reasoning, a framework that enhances large language
model (LLM) reasoning by integrating external tool-using agents. Unlike
conventional LLM-based reasoning approaches, which rely...

2. **Implementing Agents in JavaScript**
   ID: 2505.18228v1
   Authors: Timotheus Kampik
   Published: 2025-05-23
   Summary: This chapter gives an introduction to agent-oriented programming in
JavaScript. It provides an example-based walk-through of how to implement
abstractions for reasoning loop agents in vanilla JavaScri...

3. **Reasoning RAG via System 1 or System 2: A Survey on Reasoning Agentic Retrieval-Augmented Generation for Industry Challenges**
   ID: 2506.10408v1
   Authors: Jintao Liang, Gang Su, Huifeng Lin
   Published: 2025-06-12
   Summary: Retrieval-Augmented Generation (R

### Tool 2: Download Paper

In [4]:
@mcp.tool()
def download_paper(paper_id: str) -> str:
    try:
        search = arxiv.Search(id_list=[paper_id])
        paper = next(arxiv.Client().results(search))
        
        # Download PDF
        pdf_path = STORAGE_PATH / f"{paper_id}.pdf"
        paper.download_pdf(dirpath=str(STORAGE_PATH), filename=f"{paper_id}.pdf")
        
        return f"Downloaded: {paper.title}\nSaved to: {pdf_path}"
    except Exception as e:
        return f"Download error: {str(e)}"

print("✓ Added download_paper tool")


✓ Added download_paper tool


In [5]:
print("=== Testing download_paper ===")

# Use a famous paper ID (Attention is All You Need)
test_paper_id = "1706.03762"
print(f"Testing download with paper ID: {test_paper_id}")

result = download_paper(test_paper_id)
print(f"Download result: {result}")

# Check if file was created
import os
pdf_file = STORAGE_PATH / f"{test_paper_id}.pdf"
if pdf_file.exists():
    file_size = os.path.getsize(pdf_file) / (1024 * 1024)  # Size in MB
    print(f"✓ File downloaded successfully: {file_size:.1f} MB")
else:
    print("✗ File not found")

=== Testing download_paper ===
Testing download with paper ID: 1706.03762


[06/13/25 09:34:02] INFO     Requesting page (first: True, try: 0):                                 ]8;id=656188;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=791483;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#680\680]8;;\
                             https://export.arxiv.org/api/query?search_query=&id_list=1706.03762&so                
                             rtBy=relevance&sortOrder=descending&start=0&max_results=100                           

[06/13/25 09:34:03] INFO     Got first page: 1 of 1 total results                                   ]8;id=370171;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=274987;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#606\606]8;;\

Download result: Downloaded: Attention Is All You Need
Saved to: /home/siamai/deepsad/local-deep-researcher/notebooks/.arxiv-mcp/papers/1706.03762.pdf
✓ File downloaded successfully: 2.1 MB


In [7]:
# Test error handling
print("\n=== Method 3: Test error handling ===")
invalid_result = download_paper("invalid-id-123")
print(f"Invalid ID result: {invalid_result}")


=== Method 3: Test error handling ===


[06/13/25 09:37:29] INFO     Requesting page (first: True, try: 0):                                 ]8;id=639064;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=118296;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#680\680]8;;\
                             https://export.arxiv.org/api/query?search_query=&id_list=invalid-id-12                
                             3&sortBy=relevance&sortOrder=descending&start=0&max_results=100                       

[06/13/25 09:37:30] INFO     Sleeping: 2.999958 seconds                                             ]8;id=403450;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=39465;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#677\677]8;;\

[06/13/25 09:37:33] INFO     Requesting page (first: True, try: 1):                                 ]8;id=111818;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=464263;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#680\680]8;;\
                             https://export.arxiv.org/api/query?search_query=&id_list=invalid-id-12                
                             3&sortBy=relevance&sortOrder=descending&start=0&max_results=100                       

[06/13/25 09:37:34] INFO     Sleeping: 2.999969 seconds                                             ]8;id=196312;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=148280;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#677\677]8;;\

[06/13/25 09:37:37] INFO     Requesting page (first: True, try: 2):                                 ]8;id=1460;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=426711;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#680\680]8;;\
                             https://export.arxiv.org/api/query?search_query=&id_list=invalid-id-12                
                             3&sortBy=relevance&sortOrder=descending&start=0&max_results=100                       

[06/13/25 09:37:38] INFO     Sleeping: 2.999961 seconds                                             ]8;id=581352;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=74223;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#677\677]8;;\

[06/13/25 09:37:41] INFO     Requesting page (first: True, try: 3):                                 ]8;id=994969;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=839105;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#680\680]8;;\
                             https://export.arxiv.org/api/query?search_query=&id_list=invalid-id-12                
                             3&sortBy=relevance&sortOrder=descending&start=0&max_results=100                       

Invalid ID result: Download error: Page request resulted in HTTP 400 (https://export.arxiv.org/api/query?search_query=&id_list=invalid-id-123&sortBy=relevance&sortOrder=descending&start=0&max_results=100)


## Tool 3: List Papers

In [9]:
@mcp.resource("papers://downloaded")
def list_downloaded_papers() -> str:
    """List all downloaded papers"""
    papers = list(STORAGE_PATH.glob("*.pdf"))
    
    if not papers:
        return "No papers downloaded yet"
    
    return "Downloaded papers:\n" + "\n".join([
        f"• {p.stem}" for p in papers
    ])
print(list_downloaded_papers())
print("✓ Added papers resource")

[06/13/25 09:38:17] WARNING  Resource already exists: papers://downloaded                    ]8;id=700946;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/mcp/server/fastmcp/resources/resource_manager.py\resource_manager.py]8;;\:]8;id=130103;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/mcp/server/fastmcp/resources/resource_manager.py#44\44]8;;\

Downloaded papers:
• 1706.03762
✓ Added papers resource


In [10]:
def test_arxiv_server():
    print("\n=== Testing ArXiv Server ===")
    
    # Test search
    result = search_papers("reasoning model for agentic system", 5)
    print(f"Search result: {result[:100]}...")
    
    # Test resource
    papers = list_downloaded_papers()
    print(f"Downloaded papers: {papers}")

In [11]:
print("✓ ArXiv server ready")
print("Run: test_arxiv_server()")
print(test_arxiv_server())

✓ ArXiv server ready
Run: test_arxiv_server()

=== Testing ArXiv Server ===


[06/13/25 09:38:32] INFO     Requesting page (first: True, try: 0):                                 ]8;id=214903;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=292586;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#680\680]8;;\
                             https://export.arxiv.org/api/query?search_query=reasoning+model+for+ag                
                             entic+system&id_list=&sortBy=relevance&sortOrder=descending&start=0&ma                
                             x_results=100                                                                         

[06/13/25 09:38:38] INFO     Got first page: 100 of 2442103 total results                           ]8;id=881677;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py\__init__.py]8;;\:]8;id=833100;file:///home/siamai/miniconda3/envs/py310/lib/python3.10/site-packages/arxiv/__init__.py#606\606]8;;\

Search result: Found 5 papers:

1. **Agentic Reasoning: Reasoning LLMs with Tools for the Deep Research**
   ID: 25...
Downloaded papers: Downloaded papers:
• 1706.03762
None
